# Supertails Data Analyst Assessment
## B1 – Data Cleaning (Validated & Explained)

**Goal:** Prepare reliable, analysis-ready data.
Each step includes a short **why** and a visible check.

In [ ]:
import pandas as pd

### Load raw data
**Why:** Understand data size, structure, and obvious quality issues before cleaning.

In [18]:
orders = pd.read_csv('orders.csv')
support = pd.read_csv('support_tickets.csv')
vet_calls = pd.read_csv('vet_calls.csv')

print('Orders:', orders.shape)
print('Support:', support.shape)
print('Vet Calls:', vet_calls.shape)

orders.head()

Orders: (10000, 16)
Support: (5000, 8)
Vet Calls: (5000, 8)


,order_id,customer_id,order_date,order_status,payment_mode,product_id,product_category,order_value,discount_value,promised_delivery_date,actual_delivery_date,city,state,is_repeat_customer,shipment_partner,shipment_mode
0,ST2025000000,8270,2024-02-10 02:13:00,Delivered,UPI,SKU03899,Accessories,2384.14,298.90,2024-02-15 02:13:00,2024-02-14 02:13:00,Kolkata,Tamil Nadu,True,Bluedart,Surface
1,ST2025000001,1860,2024-09-09 22:45:00,Shipped,COD,SKU03852,Food,2449.95,462.15,2024-09-13 22:45:00,2024-09-14 22:45:00,Delhi,Tamil Nadu,True,Bluedart,Surface
2,ST2025000002,6390,2024-02-06 19:52:00,Delivered,COD,SKU02734,Accessories,3183.37,255.23,2024-02-09 19:52:00,2024-02-09 19:52:00,Pune,Delhi,False,XpressBees,Air
3,ST2025000003,6191,2024-10-15 18:26:00,Shipped,Wallet,SKU03811,Accessories,3814.24,139.82,2024-10-17 18:26:00,2024-10-19 18:26:00,Delhi,West Bengal,True,Ecom,Air
4,ST2025000004,6734,2024-05-14 23:22:00,Delivered,Card,SKU02654,Grooming,1519.94,192.42,2024-05-16 23:22:00,2024-05-15 23:22:00,Chennai,West Bengal,True,Bluedart,Air


### Convert date & time columns
**Why:** Correct datetime types are required for delivery, SLA, and duration metrics.

In [19]:
orders[['order_date','promised_delivery_date','actual_delivery_date']] = \
orders[['order_date','promised_delivery_date','actual_delivery_date']].apply(pd.to_datetime, errors='coerce')
support[['created_at','resolved_at']] = support[['created_at','resolved_at']].apply(pd.to_datetime, errors='coerce')
vet_calls['call_start_time'] = pd.to_datetime(vet_calls['call_start_time'], errors='coerce')

orders[['order_date','promised_delivery_date','actual_delivery_date']].dtypes

order_date                datetime64[ns]
promised_delivery_date    datetime64[ns]
actual_delivery_date      datetime64[ns]
dtype: object

### Remove invalid order timelines
**Why:** Deliveries before order date are logically incorrect and distort KPIs.

In [20]:
print('Before:', orders.shape[0])
orders_clean = orders[(orders['actual_delivery_date'].isna()) |
                       (orders['actual_delivery_date'] >= orders['order_date'])]
print('After :', orders_clean.shape[0])

orders_clean.head()

Before: 10000
After : 10000


,order_id,customer_id,order_date,order_status,payment_mode,product_id,product_category,order_value,discount_value,promised_delivery_date,actual_delivery_date,city,state,is_repeat_customer,shipment_partner,shipment_mode
0,ST2025000000,8270,2024-02-10 02:13:00,Delivered,UPI,SKU03899,Accessories,2384.14,298.90,2024-02-15 02:13:00,2024-02-14 02:13:00,Kolkata,Tamil Nadu,True,Bluedart,Surface
1,ST2025000001,1860,2024-09-09 22:45:00,Shipped,COD,SKU03852,Food,2449.95,462.15,2024-09-13 22:45:00,2024-09-14 22:45:00,Delhi,Tamil Nadu,True,Bluedart,Surface
2,ST2025000002,6390,2024-02-06 19:52:00,Delivered,COD,SKU02734,Accessories,3183.37,255.23,2024-02-09 19:52:00,2024-02-09 19:52:00,Pune,Delhi,False,XpressBees,Air
3,ST2025000003,6191,2024-10-15 18:26:00,Shipped,Wallet,SKU03811,Accessories,3814.24,139.82,2024-10-17 18:26:00,2024-10-19 18:26:00,Delhi,West Bengal,True,Ecom,Air
4,ST2025000004,6734,2024-05-14 23:22:00,Delivered,Card,SKU02654,Grooming,1519.94,192.42,2024-05-16 23:22:00,2024-05-15 23:22:00,Chennai,West Bengal,True,Bluedart,Air


### Validate support ticket timelines
**Why:** Resolution before creation gives wrong resolution-time metrics.

In [21]:
print('Before:', support.shape[0])
support_clean = support[(support['resolved_at'].isna()) |
                         (support['resolved_at'] >= support['created_at'])]
print('After :', support_clean.shape[0])

support_clean.head()

Before: 5000
After : 5000


,ticket_id,customer_id,order_id,issue_category,created_at,resolved_at,resolution_status,csat_score
0,TKT000000,5102,ST2025004600,Delivery Delay,2024-07-12 17:26:00,2024-07-18 00:26:00,Resolved,4
1,TKT000001,6947,ST2025003731,Product Issue,2024-02-03 09:47:00,2024-02-09 22:47:00,Resolved,1
2,TKT000002,3421,ST2025003998,Delivery Delay,2024-05-23 08:08:00,2024-05-26 16:08:00,Resolved,4
3,TKT000003,2319,ST2025008568,Payment Issue,2024-11-24 02:19:00,2024-11-28 00:19:00,Resolved,2
4,TKT000004,2231,ST2025004111,Delivery Delay,2024-06-15 23:49:00,2024-06-18 16:49:00,Resolved,1


### Vet call duration feature
**Why:** Minutes are easier to interpret than seconds for business reporting.

In [22]:
vet_calls_clean = vet_calls[vet_calls['call_duration_secs'] >= 0]
vet_calls_clean['call_duration_minutes'] = vet_calls_clean['call_duration_secs'] / 60

vet_calls_clean[['call_duration_secs','call_duration_minutes']].head()

,call_duration_secs,call_duration_minutes
0,1324,22.066667
1,994,16.566667
2,1499,24.983333
3,916,15.266667
4,1158,19.300000


### Final cleaned data check
**Why:** Confirms data readiness before SQL and dashboards.

In [23]:
print('Orders clean:', orders_clean.shape)
print('Support clean:', support_clean.shape)
print('Vet calls clean:', vet_calls_clean.shape)

Orders clean: (10000, 16)
Support clean: (5000, 8)
Vet calls clean: (5000, 8)


In [24]:
# Export cleaned datasets for SQL & dashboards

orders_clean.to_csv("Clean Datasets/orders_clean.csv", index=False)
support_clean.to_csv("Clean Datasets/support_tickets_clean.csv", index=False)
vet_calls_clean.to_csv("Clean Datasets/vet_calls_clean.csv", index=False)

print("Cleaned CSV files exported successfully")

Cleaned CSV files exported successfully
